In [41]:
import nltk
import networkx
from time import time
import pattern
from pattern.en import parse,pprint,parsetree
from General import TotalDistance,CrossesCountDictionary,CrossesCount
#from Rules import Rules
from itertools import permutations
import codecs
import operator

In [2]:
class Rules:
    def __init__(self):
        self.counter=0
        self.rules={}
        self.features=set()
    def AddRule(self,features):
        if len(features)<4 or u',' in features or u'.' in features:
            return
        if features in self.features:
            return
        else:
            self.features.add(features)
            for i in list(permutations([1,2,3]))[1:]:
                self.rules[self.counter]= (features,i)
                self.counter+=1

In [3]:
def BuildTree(txt):
    line = parsetree(txt,relations=True)
    #txt = [s for s in txt][0].constituents()
    wordscounter =0
    parent = Node(tag='Line')
    for s in line:
        sn = Node(tag='sentence')
        txt = s.constituents()
        for r in txt:
            if isinstance(r,pattern.text.tree.Chunk):
                ch = Node(tag=r.type)
                for w in r.words:
                    wn = Node(tag=w.type,text=w.string,index=wordscounter)
                    wordscounter+=1
                    ch.AddChildren(wn)
                sn.AddChildren(ch)
            elif isinstance(r,pattern.text.tree.Word):
                wn = Node(tag=r.type,text=r.string,index=wordscounter)
                wordscounter+=1
                sn.AddChildren(wn)
        parent.AddChildren(sn)
    return parent

In [4]:
Total = TotalDistance()

Started need around 47 seconds to finish
Finished primary Scoring


In [49]:
class Node:
    def __init__(self,tag,index=-1,text=None,parent=None):
        self.Text = text
        self.Tag= tag
        self.Parent = parent
        self.Index = index
        self.Textual= index>-1
        self.Children=[]
        
    def AddChildren(self,child):
        child.parent= self
        self.Children.append(child)
        
    def IndiciesMap(self):
        sentence=''
        indicies={}
        counter=0
        for j in self.Children:
            for i in j.Children:
                if i.Textual:
                    indicies[i.Index]=counter
                    counter+=1
                else:
                    for t in i.Children:
                        indicies[t.Index]=counter
                        counter+=1
        return indicies
    
    def GetFeature(self):
        rule = []
        rule.append(self.Children[0].Tag)
        for i in range(3):
            if i>=len(self.Children[0].Children):
                break
            rule.append(self.Children[0].Children[i].Tag)
        return tuple(rule)
    
    def ApplyRule(self,rule):
        if len(self.Children)>0:
        
            if len(self.Children[0].Children)>2:
                if sum([rule[0][i+1]==self.Children[0].Children[i].Tag for i in range(3)])==3:
                    self.Children[0].Children[:3] = [self.Children[0].Children[i-1] for i in rule[1]]
    def __str__(self):
        sentence=''
        for j in self.Children:
            for i in j.Children:
                if i.Textual:
                    sentence+= i.Text.encode('utf-8')+' '
                else:
                    for t in i.Children:
                        sentence+= t.Text.encode('utf-8')+' '
        return sentence[:-1]

In [50]:
therules = Rules()
source = open('tc-tok-all.en')
for i in range(20):
    tree = BuildTree(source.readline().strip())
    therules.AddRule(tree.GetFeature())

In [51]:
source = open('tc-tok-all.en')
alignment = open('forward.align')

In [52]:
lst =[0]*len(therules.rules)

In [53]:
for i in range(20000):
    if i%1000==0:
        print(i)
    line = source.readline().strip()
    align = alignment.readline().strip()
    try:
        for j in therules.rules.keys():
            tree = BuildTree(line)
            tree.ApplyRule(therules.rules[j])
            lst[j]+= CrossesCountDictionary(align,tree.IndiciesMap())
    except KeyError, e:
        print(i,repr(e))

In [25]:
print(lst)

[35129, 35354, 35500, 35167, 35542, 35061, 35069, 35403, 35321, 35415, 35023, 35012, 35041, 35030, 35042, 35245, 35220, 35501, 35305, 35548, 35026, 35073, 35350, 35249, 35370]


In [26]:
toprules = {}
for i in range(len(lst)):
    toprules[i]=lst[i]

In [29]:
sorted_toprule= sorted(toprules.items(), key=operator.itemgetter(1))

In [31]:
wantedRules = []
for i in range(10):
    wantedRules.append(sorted_toprule[i][0])

In [55]:
source = codecs.open('tc-tok-all.en')
output = open('re-tc-tok-all.en','w')
for i in range(5015000):
    if i%10000==0:
        print(i)
    line = source.readline().strip()
    tree = BuildTree(line)
    for i in wantedRules:
        tree.ApplyRule(therules.rules[i])
    output.write(str(tree))
    output.write('\n')

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13